# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的船票票號( 'Ticket' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
numeric_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
    else:
        numeric_features.append(feature)
print(f'{len(object_features)} object Features : {object_features}\n')
print(f'{len(numeric_features)} Numeric Features : {numeric_features}\n')

# 只留類別型欄位
df_obj = df[object_features]
df_num = df[numeric_features]
df_obj = df_obj.fillna('None')
df_num = df_num.fillna(df_num.mean())
df2 = pd.concat([df_obj,df_num],axis=1)
train_num = train_Y.shape[0]
df2.head()

5 object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,3,22.0,1,0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1,38.0,1,0,71.2833
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,3,26.0,0,0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1,35.0,1,0,53.1000
4,"Allen, Mr. William Henry",male,373450,None,S,3,35.0,0,0,8.0500


In [3]:
df2.select_dtypes(include=["object"]).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [4]:
# 加上 'Ticket' 欄位的計數編碼
count_df = df2.groupby(['Ticket'])['Name'].agg({'Ticket_Count':'size'}).reset_index()
#count_df = df.groupby(['Ticket']).agg({'Ticket_Count':'size'}).reset_index()
df2 = pd.merge(df2, count_df, on=['Ticket'], how='left')
count_df.sort_values(by=['Ticket_Count'], ascending=False).head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Ticket,Ticket_Count
778,CA. 2343,11
104,1601,8
775,CA 2144,8
335,3101295,7
454,347077,7
459,347082,7
847,S.O.C. 14879,7
824,PC 17608,7
123,19950,6
49,113781,6


In [5]:
# 'Ticket'計數編碼 + object欄位 +邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
df_temp['Ticket_Count'] = df2['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.781122155681


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,2
4,22,1,649,185,3,1


In [6]:
# 'Ticket'計數編碼 + 數值欄位 + 邏輯斯迴歸
df_temp = df_num.copy()
df_temp['Ticket_Count'] = df2['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.699299698878


,Pclass,Age,SibSp,Parch,Fare,Ticket_Count
0,3,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,2
2,3,26.0,0,0,7.9250,1
3,1,35.0,1,0,53.1000,2
4,3,35.0,0,0,8.0500,1


In [7]:
# 'Ticket'計數編碼 + object欄位 + 數值欄位 + 邏輯斯迴歸
df_temp = df2.copy()
for c in object_features:
    df_temp.drop(c,axis=1)
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
df_temp['Ticket_Count'] = df2['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.786784427334


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_Count
0,155,1,720,185,3,3,22.0,1,0,7.2500,1
1,286,0,816,106,0,1,38.0,1,0,71.2833,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,1
3,422,0,65,70,3,1,35.0,1,0,53.1000,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,1


In [8]:
# 'Ticket'特徵雜湊編碼 + object欄位 + 數值欄位 + 邏輯斯迴歸
df_temp = df2.copy()
for c in object_features:
    df_temp.drop(c,axis=1)
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
df_temp = df_temp.drop(['Ticket_Count'],axis=1)
df_temp['Ticket_Hash'] = df2['Ticket'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.787901887625


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_Hash
0,155,1,720,185,3,3,22.0,1,0,7.2500,3
1,286,0,816,106,0,1,38.0,1,0,71.2833,4
2,523,0,914,185,3,3,26.0,0,0,7.9250,9
3,422,0,65,70,3,1,35.0,1,0,53.1000,5
4,22,1,649,185,3,3,35.0,0,0,8.0500,4


In [9]:
# 'Ticket'均值編碼 + object欄位 + 數值欄位 + 邏輯斯迴歸
df_temp = pd.concat([df2[:train_num],train_Y],axis=1)
for c in object_features:
    mean_df = df_temp.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    df_temp = pd.merge(df_temp, mean_df, on=c, how='left')
    df_temp = df_temp.drop([c] , axis=1)
df_temp = df_temp.drop('Survived',axis=1)    
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

1.0


,Pclass,Age,SibSp,Parch,Fare,Ticket_Count,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,3,22.0,1,0,7.2500,1,0,0.188908,0.0,0.299854,0.336957
1,1,38.0,1,0,71.2833,2,1,0.742038,1.0,1.000000,0.553571
2,3,26.0,0,0,7.9250,1,1,0.742038,1.0,0.299854,0.336957
3,1,35.0,1,0,53.1000,2,1,0.742038,0.5,0.500000,0.336957
4,3,35.0,0,0,8.0500,1,0,0.188908,0.0,0.299854,0.336957


# 作業2
* 承上題，三者比較效果何者最好?

ANS: 均值編碼

In [10]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.786797194256


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,155,1,32,1,0,720,20,185,3
1,0,286,0,58,1,0,816,238,106,0
2,2,523,0,39,0,0,914,50,185,3
3,0,422,0,54,1,0,65,218,70,3
4,2,22,1,54,0,0,649,52,185,3


In [11]:
# 加上 'Cabin' 欄位的計數編碼
count_df2 = df2.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
#count_df = df.groupby(['Ticket']).agg({'Ticket_Count':'size'}).reset_index()
df2 = pd.merge(df2, count_df2, on=['Cabin'], how='left')
count_df2.sort_values(by=['Cabin_Count'], ascending=False).head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [12]:
# 'Cabin'計數編碼 + object欄位 + 數值欄位 + 邏輯斯迴歸
df_temp = df2.copy()
for c in object_features:
    df_temp.drop(c,axis=1)
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
df_temp['Cabin_Count'] = df2['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.792390134432


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_Count,Cabin_Count
0,155,1,720,185,3,3,22.0,1,0,7.2500,1,1014
1,286,0,816,106,0,1,38.0,1,0,71.2833,2,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,1,1014
3,422,0,65,70,3,1,35.0,1,0,53.1000,2,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,1,1014


In [13]:
# 'Cabin'特徵雜湊編碼 + object欄位 + 數值欄位 + 邏輯斯迴歸
df_temp = df2.copy()
for c in object_features:
    df_temp.drop(c,axis=1)
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
df_temp = df_temp.drop(['Cabin_Count'],axis=1)
df_temp['Cabin_Hash'] = df2['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.78455613846


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_Count,Cabin_Hash
0,155,1,720,185,3,3,22.0,1,0,7.2500,1,1
1,286,0,816,106,0,1,38.0,1,0,71.2833,2,1
2,523,0,914,185,3,3,26.0,0,0,7.9250,1,1
3,422,0,65,70,3,1,35.0,1,0,53.1000,2,7
4,22,1,649,185,3,3,35.0,0,0,8.0500,1,1


In [14]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = df2.copy()
for c in object_features:
    df_temp.drop(c,axis=1)
    df_temp[c] = LabelEncoder().fit_transform(df2[c])
df_temp['Cabin_Hash'] = df2['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.789044527122


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_Count,Cabin_Count,Cabin_Hash
0,155,1,720,185,3,3,22.0,1,0,7.2500,1,1014,1
1,286,0,816,106,0,1,38.0,1,0,71.2833,2,2,1
2,523,0,914,185,3,3,26.0,0,0,7.9250,1,1014,1
3,422,0,65,70,3,1,35.0,1,0,53.1000,2,2,7
4,22,1,649,185,3,3,35.0,0,0,8.0500,1,1014,1


# My Practice

### DataFrameGroupBy.agg(func, *args, **kwargs)
Aggregate using one or more operations over the specified axis.

* func : function, string, dictionary, or list of string/functions
    - Function to use for aggregating the data. If a function, must either work when passed a DataFrame or when passed to DataFrame.apply. For a DataFrame, can pass a dict, if the keys are DataFrame column names.

    - Accepted combinations are:
        1. string function name.
        2. function.
        3. list of functions.
        4. dict of column names -> functions (or list of functions).

In [16]:
df_test = pd.DataFrame({'A': [1, 1, 2, 2],'B': [1, 2, 3, 4],'C': np.random.randn(4)})
print('df_test = \n'+str(df_test))
df_test.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})

df_test = 
   A  B         C
0  1  1  1.280547
1  1  2  1.562995
2  2  3  1.201311
3  2  4 -0.315774


B             C
  min max       sum
A                  
1   1   2  2.843542
2   3   4  0.885537

### pandas.Series.map(arg, na_action=None)
Map values of Series using input correspondence (a dict, Series, or function).

* arg : function, dict, or Series
    - Mapping correspondence.
* na_action : {None, ‘ignore’}
    - If ‘ignore’, propagate NA values, without passing them to the mapping correspondence.

In [17]:
x_test = pd.Series([1,2,3], index=['one', 'two', 'three'])
y_test = pd.Series(['foo', 'bar', 'baz'], index=[1,2,3])
print('x_test : \n'+str(x_test)+"\n")
print('y_test : \n'+str(y_test)+"\n")
print('x_test.map(y_test) : \n'+str(x_test.map(y_test))+"\n")

z_test = {3: 'A', 1: 'B', 2: 'C'}
#z_test_pd = pd.Series([1,2,3],index=[1,2,3])
#x_test.map(z_test)
print('x_test : \n'+str(x_test.map(z_test))+"\n")

s_test = pd.Series([1, 2, 3, np.nan])
s2_test = s_test.map('this is a string {}'.format, na_action=None)
s3_test = s_test.map('this is a string {}'.format, na_action='ignore')
print('s_test : \n'+str(s_test)+"\n")
print('s2_test : \n'+str(s2_test)+"\n")
print('s3_test : \n'+str(s3_test)+"\n")

x_test : 
one      1
two      2
three    3
dtype: int64

y_test : 
1    foo
2    bar
3    baz
dtype: object

x_test.map(y_test) : 
one      foo
two      bar
three    baz
dtype: object

x_test : 
one      B
two      C
three    A
dtype: object

s_test : 
0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

s2_test : 
0    this is a string 1.0
1    this is a string 2.0
2    this is a string 3.0
3    this is a string nan
dtype: object

s3_test : 
0    this is a string 1.0
1    this is a string 2.0
2    this is a string 3.0
3                     NaN
dtype: object

